In [1]:
import cv2
import numpy as np
import pandas as pd
import os

In [2]:
rose_test_df = pd.DataFrame(columns=['name','class','x','y','w','h'])

In [11]:
img_list = os.listdir('./flower_data/test_rose/')

for f_name in img_list:
    # 이미지 로드
    img = cv2.imread(f'./flower_data/test_rose/{f_name}')

    # 이미지를 HSV 색 공간으로 변환
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # 장미꽃 봉우리의 색상 범위 정의
    lower_red = np.array([0, 50, 50])
    upper_red = np.array([10, 255, 255])

    # 임계값 처리
    mask1 = cv2.inRange(hsv, lower_red, upper_red)

    # 장미꽃 봉우리의 색상 범위 정의
    lower_red = np.array([170, 50, 50])
    upper_red = np.array([180, 255, 255])

    # 임계값 처리
    mask2 = cv2.inRange(hsv, lower_red, upper_red)

    # 두 개의 마스크를 합치기
    mask = cv2.add(mask1, mask2)

    # 블러 처리
    mask = cv2.GaussianBlur(mask, (5, 5), 0)

    # 바운딩 박스 검출
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        wh = 0
        # contour의 영역을 얻어 바운딩 박스 그리기
        x, y, w, h = cv2.boundingRect(cnt)
        if w + h > wh:
            rx, ry, rw, rh = x,y,w,h
            
    cv2.rectangle(img, (rx, ry), (rx + rw, ry + rh), (0, 255, 0), 2)

    # # 결과 이미지 출력
    # cv2.imshow('result', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    rose_test_df.loc[len(rose_test_df), :] = [f_name, 0, rx, ry, rw, rh]

In [15]:
rose_test_df

,name,class,x,y,w,h
0,0060.jpg,0,77,30,52,48
1,0062.jpg,0,0,23,173,151
2,0063.jpg,0,19,95,66,60
3,0064.jpg,0,18,16,102,98
4,0066.jpg,0,8,6,143,115
...,...,...,...,...,...,...
153,0436.jpg,0,90,15,74,63
154,0443.jpg,0,232,0,27,21
155,0450.jpg,0,48,28,159,140
156,0451.jpg,0,23,33,171,174


In [28]:
import torch
from yolov5.models.yolo import Model

# YOLOv5 모델 불러오기
model = Model(cfg="yolov5/models/yolov5m.yaml", ch=3, nc=1)  # nc는 클래스 수를 의미합니다.

# 모델 가중치를 로드하려면 다음 코드를 사용하세요.
model.load_state_dict(torch.load("yolov5m-seg.pt")['model'])

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      5280  models.common.Conv                      [3, 48, 6, 2, 2]              
  1                -1  1     41664  models.common.Conv                      [48, 96, 3, 2]                
  2                -1  2     65280  models.common.C3                        [96, 96, 2]                   
  3                -1  1    166272  models.common.Conv                      [96, 192, 3, 2]               
  4                -1  4    444672  models.common.C3                        [192, 192, 4]                 
  5                -1  1    664320  models.common.Conv                      [192, 384, 3, 2]              
  6                -1  6   2512896  models.common.C3                        [384, 384, 6]                 
  7                -1  1   2655744  models.common.Conv                      [384, 768, 3, 2]             

TypeError: Expected state_dict to be dict-like, got <class 'models.yolo.SegmentationModel'>.

In [29]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x-seg.pt', pretrained=True)

Using cache found in C:\Users\SSAFY/.cache\torch\hub\ultralytics_yolov5_master


requirements: C:\Users\SSAFY\Desktop\ \requirements.txt not found, check failed.


YOLOv5  2023-4-19 Python-3.11.2 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce RTX 2070 Super with Max-Q Design, 8192MiB)

100%|██████████| 170M/170M [01:01<00:00, 2.88MB/s] 

Fusing layers... 
YOLOv5x-seg summary: 455 layers, 88774445 parameters, 0 gradients, 265.7 GFLOPs
WARNING  YOLOv5 SegmentationModel is not yet AutoShape compatible. You will not be able to run inference with this model.


In [34]:
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 이미지 전처리 함수 정의
def preprocess(image):
    transforms = Compose([
        Resize((416, 416)),
        ToTensor(),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transforms(image)

# 예시 이미지 전처리
import PIL.Image
image = PIL.Image.open("1.jpg")
preprocessed_image = preprocess(image)

In [82]:
with torch.no_grad():
    preprocessed_image = preprocessed_image.to(device)
    output = model(preprocessed_image.unsqueeze(0))

In [61]:
from torchvision.ops import nms

threshold = 0.5  # 임계값 설정
nms_threshold = 0.4  # NMS 임계값 설정

# 임계값보다 높은 신뢰도를 가진 예측값만 선택
boxes = output["boxes"][output["scores"] > threshold]
labels = output["labels"][output["scores"] > threshold]
scores = output["scores"][output["scores"] > threshold]

# Non-Maximum Suppression (NMS) 적용
keep = nms(boxes, scores, nms_threshold)
boxes, labels, scores = boxes[keep], labels[keep], scores[keep]

IndexError: too many indices for tensor of dimension 3

In [85]:
output

[tensor([[[ 4.32382e+00,  8.55536e+00,  8.17431e+00,  ..., -1.26839e+00, -1.22777e-01, -3.62971e-01],
          [ 6.98499e+00,  6.82913e+00,  1.21310e+01,  ..., -1.11879e+00, -2.14462e-01, -2.95892e-01],
          [ 1.99491e+01,  6.31583e+00,  1.78380e+01,  ..., -7.18749e-01, -1.82613e-01, -5.33644e-01],
          ...,
          [ 3.48301e+02,  3.80058e+02,  1.31779e+02,  ..., -5.50191e-01,  2.33497e-01, -7.45418e-01],
          [ 3.61868e+02,  3.79901e+02,  1.21663e+02,  ..., -4.10262e-01,  8.54912e-02, -7.29382e-01],
          [ 3.96372e+02,  3.88045e+02,  1.41481e+02,  ..., -4.08033e-01,  1.01068e-01, -6.36469e-01]]], device='cuda:0'),
 tensor([[[[-0.15137, -0.21736, -0.21426,  ..., -0.08556, -0.09114, -0.07784],
           [-0.17768, -0.26473, -0.26184,  ...,  0.17860,  0.05227,  0.03980],
           [-0.20311, -0.27372, -0.27242,  ...,  0.23736,  0.11796,  0.05240],
           ...,
           [-0.24803, -0.27666, -0.27661,  ...,  0.13252,  0.16529,  0.04630],
           [-0.22503,